# 2020a_4DVARQG
This notebook, associated to the configuration file *config_2020a_4DVARQG.py* runs a BFN-QG experiment to map SSH from simulated SWOT and Nadir data

## Configuration file

In [1]:
path_config = 'config_2020a_4DVARQG'

In [2]:
import sys
sys.path.append('..')

In [3]:

from src import exp
config = exp.Exp(path_config)

name_experiment: 2020a_4DVARQG_test
saveoutputs: True
name_exp_save: 2020a_4DVARQG_test
path_save: ../outputs/2020a_4DVARQG/2020a_4DVARQG_test
tmp_DA_path: ../scratch/2020a_4DVARQG/2020a_4DVARQG_test
init_date: 2012-10-01 00:00:00
final_date: 2012-10-15 00:00:00
assimilation_time_step: 6:00:00
saveoutput_time_step: 6:00:00
flag_plot: 1
name_lon: lon
name_lat: lat
name_time: time
plot_time_step: 1 day, 0:00:00
time_obs_min: None
time_obs_max: None
write_obs: False
compute_obs: False
path_obs: None



## State

In [4]:
from src import state as state
State = state.State(config)

super: GRID_CAR
lon_min: 295.0
lon_max: 305.0
lat_min: 33.0
lat_max: 43.0
dx: 25.0
name_init_mask: None
name_var_mask: {'lon': '', 'lat': '', 'var': ''}



## Model

In [ ]:
from src import mod as mod
Model = mod.Model(config,State)

In [ ]:
from src import bc as bc
Bc = bc.Bc(config)

## Observations

In [ ]:
from src import obs as obs
dict_obs = obs.Obs(config,State)

In [ ]:
from src import obsop as obsop
Obsop = obsop.Obsop(config,State,dict_obs,Model)

## Reduced Basis

In [ ]:
from src import basis as basis
Basis = basis.Basis(config,State)

## Assimilation

In [ ]:
from src import inv as inv
inv.Inv(config,State,Model,dict_obs=dict_obs,Obsop=Obsop,Basis=Basis,Bc=Bc)

## Diagnostics

In [5]:
from src import diag as diag
Diag = diag.Diag(config,State)

super: DIAG_OSSE
dir_output: None
time_min: None
time_max: None
lon_min: None
lon_max: None
lat_min: None
lat_max: None
name_ref: ../../data/2020a_SSH_mapping_NATL60/dc_ref/NATL60-CJM165_GULFSTREAM*.nc
name_ref_time: time
name_ref_lon: lon
name_ref_lat: lat
name_ref_var: sossheig
options_ref: {'combine': 'nested', 'concat_dim': 'time', 'parallel': True}
name_exp_var: ssh



In [6]:
Diag.regrid_exp()

ValueError: points must be a 1-dimensional array

In [14]:
from scipy import interpolate 
import pyinterp
import numpy as np

lon = +Diag.exp[Diag.name_exp_lon].values
lon = lon[~np.isnan(lon)]

lat = +Diag.exp[Diag.name_exp_lat].values
lat = lat[~np.isnan(lat)]

y_source_axis = pyinterp.Axis(lat)
x_source_axis = pyinterp.Axis(lon, is_circle=False)
y_source_axis = pyinterp.Axis(lat)
z_source_axis = pyinterp.TemporalAxis(Diag.exp[Diag.name_exp_time][:].values)


ValueError: axis values are not ordered

In [12]:
lon


array([295.        , 295.27099166, 295.54198331, ..., 304.63344348,
       304.94419972, 305.25495596])

In [8]:
Diag.exp.ssh

<xarray.DataArray 'ssh' (time: 57, y: 45, x: 38)>
dask.array<concatenate, shape=(57, 45, 38), dtype=float64, chunksize=(1, 45, 38), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2012-10-01 2012-10-01T06:00:00 ... 2012-10-15
    lon      (y, x) float64 dask.array<chunksize=(45, 38), meta=np.ndarray>
    lat      (y, x) float64 dask.array<chunksize=(45, 38), meta=np.ndarray>
Dimensions without coordinates: y, x

In [ ]:
Diag.rmse_based_scores(plot=True)

In [ ]:
Diag.psd_based_scores(plot=True)

In [ ]:
Diag.movie(framerate=12)

In [ ]:
import os
from IPython.display import Video
Video(os.path.join(Diag.dir_output, 'movie.mp4'))

In [ ]:
Leaderboard = Diag.Leaderboard()
Leaderboard